# `Jai Chaudhry 2K18-SE-069`

## Experiment - 4

- Write a program to demonstrate the working of the decision tree based C4.5 algorithm. 
- With the help of data set used in above experiment build the decision tree and classify a new sample. 
- Then Finding Accuracy, Precision, Recall and ROC_AUC Score of the Decision Tree

In [573]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log


- `eps` here is the smallest representable number. 
- At times we get log(0) or 0 in the denominator, to avoid that we are going to use this.

# Define the Dataset : (Class Dataset)

In [574]:
outlook = 'sunny,sunny,overcast,rain,rain,rain,overcast,sunny,sunny,rain,sunny,overcast,overcast,rain'.split(',')
temp='hot,hot,hot,mild,cold,cold,cold,mild,cold,mild,mild,mild,hot,mild'.split(',')
humidity = 'high,high,high,high,normal,normal,normal,high,normal,normal,normal,high,normal,high'.split(',')
windy = 'weak,strong,weak,weak,weak,strong,strong,weak,weak,weak,strong,strong,weak,strong'.split(',')
play = 'no,no,yes,yes,yes,no,yes,no,yes,yes,yes,yes,yes,no'.split(',')


In [575]:
dataset=0

dataset = {'outlook':outlook,'temp':temp,'humidity':humidity,'windy':windy,'play':play}
df = pd.DataFrame(dataset,columns=['outlook','temp','humidity','windy','play'])


In [576]:
df.head(14)


,outlook,temp,humidity,windy,play
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rain,mild,high,weak,yes
4,rain,cold,normal,weak,yes
5,rain,cold,normal,strong,no
6,overcast,cold,normal,strong,yes
7,sunny,mild,high,weak,no
8,sunny,cold,normal,weak,yes
9,rain,mild,normal,weak,yes


## Decision Tree Algorithm
- compute the entropy for data-set
- for every attribute/feature:
       1.calculate entropy for all categorical values
       2.take average information entropy for the current attribute
       3.calculate split info for the current attribute
       4.calculate gain for the current attribute
- pick the highest gain attribute.
- Repeat until we get the tree we desired

## Get Entropy of DataFrame `Df` for a subtree:

In [577]:
# ok 
def find_entropy(df): 
    Class = df.keys()[-1]           # To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction * np.log2(fraction)
    return entropy


In [578]:
print(log(2))


1.0


# Find entropy of df wrt a certain attribute

In [579]:
# ok
def find_entropy_attribute(df,attribute):
    Class = df.keys()[-1]                  # To make the code generic, changing target variable class name
#     print(Class)
    
    target_variables = df[Class].unique()  # This gives all 'Yes' and 'No'
    variables = df[attribute].unique()     # This gives different features in that attribute (like 'Hot','Cold' in Temperature)
    entropy2 = 0
    
    for variable in variables:
        entropy = 0
        den = len(df[attribute][df[attribute]==variable])
        
        for target_variable in target_variables:
            num = len(df[attribute][df[attribute]==variable][df[Class] == target_variable])
            fraction = num/(den+eps)
            entropy += -fraction*log(fraction+eps)
            
        fraction2 = den/len(df)
        entropy2 += fraction2*entropy
    
    return abs(entropy2)


In [580]:
for i in df.columns: 
    print(i,": ",find_entropy_attribute(df,i))


outlook :  0.6935361388961914
temp :  0.9110633930116756
humidity :  0.7884504573082889
windy :  0.892158928262361
play :  3.2034265038149176e-16


# Find Split Info. for a Particular Attribute

In [581]:
# OK 
def get_split_info(df,attr):
    variables = df[attr].unique()
    
    split_info = 0.0
    den = len(df[attr])
    
    for val in variables:
    
        num = len( df[attr][df[attr]==val] )
#         print(val,num)
        
        fraction = num / den
        split_info += - fraction * log(fraction + eps)
    
    return split_info


In [582]:
# OK
get_split_info(df,'outlook')


1.5774062828523445

# Get The one that is the best divider for current node

In [583]:
# ok   
def find_winner(df):
   
    Entropy_att = []
    IG = []
    
    for key in df.keys()[:-1]:
        gain = find_entropy(df) - find_entropy_attribute(df,key)
        gain = gain / ( get_split_info(df,key) + eps)
        IG.append(gain)

    return df.keys()[:-1][np.argmax(IG)]                                


In [584]:
find_winner(df)


'outlook'

# After choosing an Attr. for division, 
### For a particular branch from this attr. for a specific value x 
### Get the subset of data that has this attr.'s value as x 

In [585]:
def get_subtable(df, node ,value): 
    return df[df[node] == value].reset_index(drop=True) 


In [586]:
df.drop(['play'],axis=1).head(3)


,outlook,temp,humidity,windy
0,sunny,hot,high,weak
1,sunny,hot,high,strong
2,overcast,hot,high,weak


# Main Driver Function to create Tree

In [587]:
def buildTree(df,TgtClass,previous_winner=0,tree=None): 
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    
    # Here we build our decision tree

    # Get attribute with maximum information gain
    node = find_winner(df)
    
    print(node)
    
    # Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
    
    if previous_winner == node:
        clValue,counts = np.unique(df[TgtClass],return_counts=True) 
        tree[node][df[node][0]] = clValue[np.argmax(counts)]
        return tree
    
    
    # Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(df[node])

   # We make loop to construct a tree by calling this function recursively. 
   # In this we check if the subset is pure and stops if it is pure. 

    for value in attValue:
        
        subtable = get_subtable(df,node,value)
        clValue,counts = np.unique(subtable[TgtClass],return_counts=True)   
        
        if len(counts)==1:                # Checking purity of subset
            tree[node][value] = clValue[0]   
        else:        
            tree[node][value] = buildTree(subtable,TgtClass,node) #Calling the function recursively 
                   
    return tree


## Running It 

In [588]:
t = buildTree(df,'play')

outlook
windy
humidity


In [589]:
import pprint      
pprint.pprint(t)    


{'outlook': {'overcast': 'yes',
             'rain': {'windy': {'strong': 'no', 'weak': 'yes'}},
             'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}


In [590]:
graph = pydot.Dot(graph_type='graph')
visit(t)
graph.write_png('Play_Tennis_C4.5.png')


<img src="Play_Tennis_C4.5.png" >

# Now Implementing Decision Tree on Adult DataSet

# Loading Data

In [591]:
import copy 
import pandas as pd
data = pd.read_csv('C:\\Users\\Jai\\Desktop\\adult.csv',na_values='?')

num_cols = list(data.select_dtypes(include=["number"]).columns)
cat_cols = list(data.select_dtypes(exclude=["number"]).columns)

print((num_cols))
print()
print(cat_cols)


['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']

['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country', 'income']


# PreProcessing

In [592]:
print(len(data))
data = data.dropna() 
print(len(data))     
data.head()       


48842
45222


,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


# Getting Numerical and Categorical Columns 


In [593]:
# RUN THIS ONLY ONCE

# Numeric Data
ndata = data.select_dtypes(include=['number']) 
indata = copy.deepcopy(ndata)

# Categorical Data
cdata = data.select_dtypes(exclude=['number']) 

print(len(data))
cdata.head(10)     

data.describe()

45222


,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
count,45222.000000,4.522200e+04,45222.000000,45222.000000,45222.000000,45222.000000
mean,38.547941,1.897347e+05,10.118460,1101.430344,88.595418,40.938017
std,13.217870,1.056392e+05,2.552881,7506.430084,404.956092,12.007508
min,17.000000,1.349200e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.173882e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783160e+05,10.000000,0.000000,0.000000,40.000000
75%,47.000000,2.379260e+05,13.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


In [594]:
# Numeric Data
ndata =  copy.deepcopy(indata)
ndata.head()


,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
0,25,226802,7,0,0,40
1,38,89814,9,0,0,50
2,28,336951,12,0,0,40
3,44,160323,10,7688,0,40
5,34,198693,6,0,0,30


# Converting Numeric Values to 0 or 1 values for Decision tree (Using Mean)

In [595]:
# a = np.array(ndata,dtype=object)
# print(a.shape)
# print(a[0])
# a[0][0] = str(23) + "s"
# print(a[0][0])

ndata =  copy.deepcopy(indata)

cols = ndata.columns
b = np.array(ndata)
a = np.array(ndata,dtype=object)
print(a.shape)

for i in range(0,len(cols)):
    
    col = cols[i]
    meann = int(np.round(ndata[col].mean()))
    print(col,meann)
    
    for j in range(0,len(ndata)):
        if b[j][i] >= meann:
            a[j][i] = ">=" + str(meann)
        else:
            a[j][i] = "<" + str(meann)


newdata = pd.DataFrame(a).applymap(str)
ndata=newdata
ndata.columns = cols

print(len(ndata))
newdata.head(10)


(45222, 6)
age 39
fnlwgt 189735
educational-num 10
capital-gain 1101
capital-loss 89
hours-per-week 41
45222


,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
0,<39,>=189735,<10,<1101,<89,<41
1,<39,<189735,<10,<1101,<89,>=41
2,<39,>=189735,>=10,<1101,<89,<41
3,>=39,<189735,>=10,>=1101,<89,<41
4,<39,>=189735,<10,<1101,<89,<41
5,>=39,<189735,>=10,>=1101,<89,<41
6,<39,>=189735,>=10,<1101,<89,<41
7,>=39,<189735,<10,<1101,<89,<41
8,>=39,<189735,<10,>=1101,<89,<41
9,<39,>=189735,>=10,<1101,<89,<41


In [596]:
# ## Assigning categorical values (like interval) to numeric data
# ## Again run this only once

# ndata =  copy.deepcopy(indata)
# print(ndata.head())


# for i in ndata.columns:
    
#     mn = ndata[i].mean()
#     print(i,mn)
    
# #     ndata[i] = ndata[i].apply(str)

# #     ndata[i][ndata[i]<mn] = 0
# #     ndata[i][ndata[i]>=mn] = 1

# #     ndata.loc[(ndata[i]) < mn, i] = 0
# #     ndata.loc[(ndata[i]) >= mn, i] = 1
    
# newdata.head()

for col in cols:
    print(ndata[col].value_counts())
    print()


<39     24238
>=39    20984
Name: age, dtype: int64

<189735     25426
>=189735    19796
Name: fnlwgt, dtype: int64

>=10    24778
<10     20444
Name: educational-num, dtype: int64

<1101     41534
>=1101     3688
Name: capital-gain, dtype: int64

<89     43082
>=89     2140
Name: capital-loss, dtype: int64

<41     31445
>=41    13777
Name: hours-per-week, dtype: int64



# Merging the Transformed Numeric Data with Categorical Data

In [597]:
ndata.reset_index(drop=True, inplace=True)
cdata.reset_index(drop=True,inplace=True)
frames = [ndata,cdata]
print(len(ndata),len(cdata))


45222 45222


In [598]:
result = pd.concat(frames,axis = 1)
data =  result

print(len(data))

data.head()


45222


,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass,education,marital-status,occupation,relationship,race,gender,native-country,income
0,<39,>=189735,<10,<1101,<89,<41,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,United-States,<=50K
1,<39,<189735,<10,<1101,<89,>=41,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,United-States,<=50K
2,<39,>=189735,>=10,<1101,<89,<41,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,United-States,>50K
3,>=39,<189735,>=10,>=1101,<89,<41,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,United-States,>50K
4,<39,>=189735,<10,<1101,<89,<41,Private,10th,Never-married,Other-service,Not-in-family,White,Male,United-States,<=50K


# All set to Run on the Dataset

In [610]:
number_of_tuples_i_want = 30000 

npdata = np.array(data)  
print(npdata.shape)   

(45222, 15)


In [611]:
newdata = data[:min(number_of_tuples_i_want,len(data))]     
print(len(newdata))

newdata.head(5)

30000


,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass,education,marital-status,occupation,relationship,race,gender,native-country,income
0,<39,>=189735,<10,<1101,<89,<41,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,United-States,<=50K
1,<39,<189735,<10,<1101,<89,>=41,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,United-States,<=50K
2,<39,>=189735,>=10,<1101,<89,<41,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,United-States,>50K
3,>=39,<189735,>=10,>=1101,<89,<41,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,United-States,>50K
4,<39,>=189735,<10,<1101,<89,<41,Private,10th,Never-married,Other-service,Not-in-family,White,Male,United-States,<=50K


In [612]:
print(newdata['income'][0])
# newdata.head()

<=50K


# Convert The Dataframe values to string so that it can be plotted using pydot

In [613]:
newdata = newdata.applymap(str)

In [614]:
tree = buildTree(newdata,'income')  


capital-gain
marital-status
hours-per-week
educational-num
capital-loss
native-country
education
occupation
relationship
workclass
age
race
fnlwgt
age
age
age
age
gender
race
age
fnlwgt
age
age
fnlwgt
education
age
age
age
relationship
gender
education
workclass
race
fnlwgt
age
age
age
age
education
workclass
fnlwgt
race
age
relationship
age
age
relationship
gender
fnlwgt
age
age
age
age
relationship
fnlwgt
gender
age
relationship
education
age
age
workclass
education
fnlwgt
gender
age
fnlwgt
workclass
age
age
workclass
race
age
age
relationship
fnlwgt
relationship
age
education
race
age
age
age
relationship
fnlwgt
education
workclass
age
age
occupation
fnlwgt
workclass
age
age
age
fnlwgt
age
age
fnlwgt
native-country
age
fnlwgt
gender
age
occupation
education
workclass
workclass
occupation
fnlwgt
education
occupation
relationship
capital-loss
fnlwgt
workclass
age
race
age
age
gender
fnlwgt
workclass
relationship
race
age
workclass
relationship
race
workclass
age
capital-loss
occupatio

age
age
age
age
occupation
relationship
fnlwgt
age
age
fnlwgt
age
age
age
age
relationship
fnlwgt
age
age
age
age
relationship
fnlwgt
age
age
age
age
fnlwgt
age
age
fnlwgt
age
age
fnlwgt
age
age
age
age
fnlwgt
relationship
age
age
age
age
relationship
age
age
relationship
fnlwgt
age
age
age
age
fnlwgt
age
age
fnlwgt
age
age
relationship
fnlwgt
age
age
age
age
relationship
fnlwgt
age
age
age
age
occupation
fnlwgt
hours-per-week
age
age
hours-per-week
age
age
age
age
hours-per-week
fnlwgt
age
age
fnlwgt
age
age
age
age
fnlwgt
age
age
age
age
occupation
relationship
hours-per-week
fnlwgt
age
age
age
age
fnlwgt
age
age
hours-per-week
fnlwgt
age
age
age
age
fnlwgt
hours-per-week
age
age
relationship
hours-per-week
fnlwgt
age
age
age
age
fnlwgt
hours-per-week
age
age
fnlwgt
hours-per-week
age
age
age
age
hours-per-week
age
age
age
age
occupation
fnlwgt
age
age
hours-per-week
age
age
fnlwgt
fnlwgt
hours-per-week
relationship
age
age
occupation
workclass
fnlwgt
hours-per-week
race
relationship

race
workclass
age
age
age
hours-per-week
fnlwgt
age
age
age
age
fnlwgt
age
age
relationship
age
age
age
age
relationship
fnlwgt
hours-per-week
age
age
age
age
hours-per-week
age
age
age
hours-per-week
age
age
age
age
fnlwgt
hours-per-week
age
age
age
age
workclass
hours-per-week
fnlwgt
age
race
age
age
fnlwgt
hours-per-week
relationship
age
age
relationship
age
age
age
relationship
age
hours-per-week
age
age
hours-per-week
age
age
fnlwgt
hours-per-week
age
age
age
age
hours-per-week
age
age
fnlwgt
hours-per-week
fnlwgt
relationship
workclass
race
hours-per-week
age
fnlwgt
age
age
fnlwgt
age
age
age
age
fnlwgt
race
age
age
hours-per-week
age
age
age
race
hours-per-week
fnlwgt
age
age
age
age
fnlwgt
age
age
age
age
race
fnlwgt
hours-per-week
age
age
hours-per-week
fnlwgt
age
age
age
age
fnlwgt
age
age
age
age
race
hours-per-week
age
fnlwgt
age
age
fnlwgt
age
age
age
age
age
fnlwgt
age
age
fnlwgt
age
age
age
age
age
hours-per-week
age
age
hours-per-week
fnlwgt
age
age
age
age
fnlwgt
age


age
age
age
age
hours-per-week
age
age
age
age
fnlwgt
hours-per-week
age
age
workclass
hours-per-week
fnlwgt
age
age
age
age
fnlwgt
age
age
hours-per-week
age
fnlwgt
age
age
fnlwgt
age
age
fnlwgt
workclass
age
hours-per-week
race
age
age
fnlwgt
workclass
age
age
age
age
age
age
age
workclass
age
age
age
age
age
age
age
race
age
workclass
workclass
age
fnlwgt
age
age
age
age
fnlwgt
age
age
age
age
age
fnlwgt
age
age
fnlwgt
age
age
age
age
fnlwgt
age
age
age
age
age
age
race
hours-per-week
education
workclass
age
fnlwgt
age
age
fnlwgt
age
age
age
age
fnlwgt
workclass
age
fnlwgt
age
age
age
age
fnlwgt
age
age
age
age
fnlwgt
age
workclass
education
age
age
age
education
native-country
education
race
age
education
age
relationship
relationship
age
fnlwgt
workclass
hours-per-week
education
hours-per-week
hours-per-week
age
age
age
age
hours-per-week
hours-per-week
age
age
age
age
hours-per-week
age
age
workclass
hours-per-week
education
age
age
education
hours-per-week
age
age
hours-per-week

age
education
fnlwgt
workclass
race
fnlwgt
workclass
relationship
occupation
gender
age
age
occupation
fnlwgt
native-country
education
relationship
occupation
workclass
fnlwgt
capital-loss
educational-num
native-country
age
gender
relationship
fnlwgt
occupation
age
age
occupation
relationship
workclass
education
race
age
fnlwgt
age
fnlwgt
hours-per-week
education
workclass
relationship
age
age
age
age
fnlwgt
workclass
workclass
fnlwgt
education
workclass
occupation
relationship
occupation
native-country
workclass
occupation
hours-per-week
workclass
relationship
age
race
workclass
fnlwgt
occupation
fnlwgt
relationship
gender
age
age
fnlwgt
age
age
fnlwgt
occupation
relationship
race
native-country
workclass
fnlwgt
age
occupation
fnlwgt
gender
occupation
workclass
fnlwgt
hours-per-week
fnlwgt
workclass
fnlwgt
workclass
age
age
workclass
gender
fnlwgt
age
gender
educational-num
race
gender
workclass
relationship
native-country
occupation
age
age
capital-loss
hours-per-week
education
workc

# Saving Our Decision Tree

In [606]:
import pickle 

with open('C45.pickle', 'wb') as handle: 
    pickle.dump(tree, handle, protocol=pickle.HIGHEST_PROTOCOL) 

with open('C45.pickle', 'rb') as handle:
    b = pickle.load(handle)
    
tree == b

True

# Visualizing Decision Tree

In [607]:
import pydot

dict = {}

menu = {'dinner':
            {'chicken':'good',
             'beef':'average',
             'vegetarian':{
                   'tofu':'good',
                   'salad':{
                            'caeser':'bad',
                            'italian':'average'}
                   },
             'pork':'bad'}
        }


def draw(parent_name, child_name):
    edge = pydot.Edge(parent_name, child_name)
    graph.add_edge(edge)


def visit(node, parent=None):
    for k,v in node.items():
        if isinstance(v, type(dict)):
            # We start with the root node whose parent is None
            # we don't want to graph the None node
            if parent:
                draw(parent, k)
            visit(v, k)
            
        else:
            draw(parent, k)
            # drawing the label using a distinct name
            draw(k, k+'_'+v)


graph = pydot.Dot(graph_type='graph')

# visit(menu)
visit(tree)
# visit(t)
# print(tree)

graph.write_png('Adult_Dataset_C4.5.png')

<img src = "Adult_Dataset_C4.5.png"> 

# Classifying a new sample

In [615]:
limit = 2*14

err = 0
exception = 0
returning = 0

def predict(tree,prev,sample,cnt):
    
    # Find the current attr / parameter
    # Check the value of that in sample 
    # Then navigate through the tree accordingly 
    
    global err, exception, returning
    
    if cnt > limit:
#         print(cnt>limit) 
        err += 1
        return "idk"
    try:
        tkeys = list(tree.keys())
    except:
#         print(tree) 
        return tree
    
    try:
        if len(tkeys)==1:
            # Found a category 
            t=1
            return predict(tree[tkeys[0]],tkeys[0],sample,cnt+1)
        else:
    #         print("GOING WITH ",sample[prev],"\n") 
            return predict(tree[sample[prev]],"-1",sample,cnt+1)
    except:
        exception += 1
        return "idk"
    
    returning += 1
    return "idk"


In [616]:
err = 0
exception = 0
returning = 0

sample = data.iloc[1422].to_dict()
# pprint.pprint(sample)

print("Actual : ",sample['income'])

print("Prediction : ",predict(tree,"-1",sample,0) )

cnt = 0
for i in range(0,45000):
    sample = data.iloc[i].to_dict()
    pred = predict(tree,"-1",sample,0)
    
    if pred == "idk":
        cnt+=1

print(cnt)
print(err)
print(exception)
print(returning)


Actual :  <=50K
Prediction :  <=50K
10226
9573
653
0


# Get Prediction for a Certain Size

In [617]:
size = 45000

y_actual = data['income'].iloc[0:size]
y_actual = np.array(y_actual)

print(y_actual.shape)

preds = np.array((y_actual),dtype=object)
for i in range(0,size):
    try:
        preds[i] = predict(tree,"-1",data.iloc[i].to_dict(),0)
    except:
        preds[i] = "idk"


(45000,)


# Convert to Binary

In [618]:
testy = y_actual
yhat_classes = preds

for i in range(testy.shape[0]):
    if testy[i]=='<=50K':
        testy[i]='0'

    elif testy[i]=='>50K':
        testy[i]='1'


    if yhat_classes[i]=='<=50K':
        yhat_classes[i]='0'

    elif yhat_classes[i]=='>50K':
        yhat_classes[i]='1'
        
    elif yhat_classes[i]=='idk':
        yhat_classes[i] = str((1-int(testy[i])))


testy = np.array(testy,dtype=int)
yhat_classes = np.array(yhat_classes,dtype=int)

print(testy.dtype,"\n")

temp = pd.DataFrame(yhat_classes)
temp[0].value_counts()

int32 



0    32141
1    12859
Name: 0, dtype: int64

# Calculating  _  Accuracy | Precision | Recall | F1 score | ROC_AUC_Score

In [619]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(testy, yhat_classes)*100
print('Accuracy      : %f' % (accuracy),"%")


# precision tp / (tp + fp)
precision = precision_score(testy, yhat_classes)*100
print('Precision     : %f' % precision,"%")


# recall: tp / (tp + fn)
recall = recall_score(testy, yhat_classes)*100
print('Recall        : %f' % recall,"%")


# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(testy, yhat_classes)*100
print('F1 score      : %f' % f1,"%")
 

# ROC AUC
auc = roc_auc_score(testy, yhat_classes)*100
print("roc_auc_score :",auc,"%")

Accuracy      : 71.368889 %
Precision     : 43.308189 %
Recall        : 49.888023 %
F1 score      : 46.365831 %
roc_auc_score : 64.17177988563776 %


<img src = "Adult_Dataset_C4.5.png"> 

# `Jai Chaudhry 2K18-SE-069`